In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 29 05:34:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [179]:
import json
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import sklearn.preprocessing as preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain
from sklearn.preprocessing import StandardScaler

from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD 



#embedding
from gensim.models import Word2Vec

# tqdm
from tqdm.notebook import tqdm



In [47]:
train_file_path = "/content/drive/MyDrive/assign2/comp90051-22-s2-authorship/train.json"
test_file_path = "/content/drive/MyDrive/assign2/comp90051-22-s2-authorship/test.json"

with open(train_file_path, 'r') as f:
    data = json.load(f)
df_train = pd.DataFrame(data)

with open(test_file_path, 'r') as f:
    data = json.load(f)
df_test = pd.DataFrame(data)

In [48]:
# Retrive the non-prolific author and prolific author index
non_prolific_author_index = []
prolific_author_index = []
for i,j in enumerate(df_train['authors']):
  sum = 0
  for k in j:
    if k < 100 and k >= 0:
      sum += 1
  if sum == 0:
    non_prolific_author_index.append(i)
  else:
    prolific_author_index.append(i)

df_non_prolifc_authors = df_train.drop(prolific_author_index)
df_prolific_authors = df_train.drop(non_prolific_author_index)

In [49]:
df_prolific_authors

,authors,year,abstract,venue,title
0,"[42, 13720, 36]",9,"[2455, 1858, 2335, 1543, 1800, 1860, 2000, 286...",20,"[41, 1550, 1563, 1594, 1544, 1919, 1644, 37, 1..."
1,"[1359, 15881, 45]",15,"[40, 1542, 1691, 2449, 1535, 3616, 2206, 1904,...",2,"[1731, 47, 11, 57, 4624, 1525, 1535, 47, 11, 3..."
3,[97],10,"[46, 1624, 1547, 56, 1687, 1644, 6, 7, 3386, 1...",4,"[40, 1733, 1735, 1540, 1655, 46, 1624, 1547, 5..."
4,"[19617, 2]",10,"[37, 3709, 3836, 1586, 2151, 1727, 3021, 1860,...",9,"[38, 1592, 2088, 1543, 1574, 1727, 1597, 1813,..."
9,"[9641, 44, 5623, 2]",18,"[1731, 2021, 1543, 11, 1546, 11, 1647, 2163, 1...",0,"[1560, 1694, 11, 1546, 11, 3066, 1728, 47, 160..."
...,...,...,...,...,...
25767,"[19974, 68, 8903]",1,"[40, 1542, 1691, 2449, 1535, 2855, 1549, 1755,...",23,"[1708, 33, 58, 57, 2174, 1548, 1538, 1553, 359..."
25776,"[6353, 67, 3037, 15856, 13521]",10,"[40, 1542, 1691, 2907, 1540, 1535, 2121, 2431,...",10,"[53, 2555, 4480, 1535, 1637, 1898, 1549, 53, 3..."
25778,"[7686, 3810, 87, 7451]",16,"[2144, 1661, 2151, 1966, 3776, 4864, 1546, 219...",31,"[1615, 1966, 11, 3238, 1653, 3318, 24, 2144, 1..."
25781,"[14864, 92]",16,"[37, 1662, 33, 1814, 2036, 1692, 1553, 2333, 2...",0,"[46, 1670, 1826, 37, 1632, 46, 1991, 1525, 152..."


In [50]:
# Retrieve the prolific author as label
prolific_author_label = []
non_prolific_author_label = []
for i in df_prolific_authors['authors']:
  prolific_author_label_each_instance = []
  non_prolific_author_label_each_instance = []
  for j in i:
    if j >= 0 and j < 100:
      prolific_author_label_each_instance.append(j)
    else:
      non_prolific_author_label_each_instance.append(j)
  prolific_author_label.append(prolific_author_label_each_instance)
  non_prolific_author_label.append(non_prolific_author_label_each_instance)


df_prolific_authors['coauthors'] = non_prolific_author_label

In [51]:
#One-hot encoding the labels
multilable_binarizer = MultiLabelBinarizer()
ohe_prolific_labels = multilable_binarizer.fit_transform(prolific_author_label).astype('float')

In [12]:
# df_prolific_authors['sentences'] = df_prolific_authors['title'] + df_prolific_authors['abstract'] 
# df_prolific_authors = df_prolific_authors.drop(['abstract','title'],axis = 1)

# df_test['sentences'] = df_test['title'] + df_test['abstract'] 
# df_test = df_test.drop(['abstract','title'],axis = 1)

In [52]:
df_prolific_authors

,authors,year,abstract,venue,title,coauthors
0,"[42, 13720, 36]",9,"[2455, 1858, 2335, 1543, 1800, 1860, 2000, 286...",20,"[41, 1550, 1563, 1594, 1544, 1919, 1644, 37, 1...",[13720]
1,"[1359, 15881, 45]",15,"[40, 1542, 1691, 2449, 1535, 3616, 2206, 1904,...",2,"[1731, 47, 11, 57, 4624, 1525, 1535, 47, 11, 3...","[1359, 15881]"
3,[97],10,"[46, 1624, 1547, 56, 1687, 1644, 6, 7, 3386, 1...",4,"[40, 1733, 1735, 1540, 1655, 46, 1624, 1547, 5...",[]
4,"[19617, 2]",10,"[37, 3709, 3836, 1586, 2151, 1727, 3021, 1860,...",9,"[38, 1592, 2088, 1543, 1574, 1727, 1597, 1813,...",[19617]
9,"[9641, 44, 5623, 2]",18,"[1731, 2021, 1543, 11, 1546, 11, 1647, 2163, 1...",0,"[1560, 1694, 11, 1546, 11, 3066, 1728, 47, 160...","[9641, 5623]"
...,...,...,...,...,...,...
25767,"[19974, 68, 8903]",1,"[40, 1542, 1691, 2449, 1535, 2855, 1549, 1755,...",23,"[1708, 33, 58, 57, 2174, 1548, 1538, 1553, 359...","[19974, 8903]"
25776,"[6353, 67, 3037, 15856, 13521]",10,"[40, 1542, 1691, 2907, 1540, 1535, 2121, 2431,...",10,"[53, 2555, 4480, 1535, 1637, 1898, 1549, 53, 3...","[6353, 3037, 15856, 13521]"
25778,"[7686, 3810, 87, 7451]",16,"[2144, 1661, 2151, 1966, 3776, 4864, 1546, 219...",31,"[1615, 1966, 11, 3238, 1653, 3318, 24, 2144, 1...","[7686, 3810, 7451]"
25781,"[14864, 92]",16,"[37, 1662, 33, 1814, 2036, 1692, 1553, 2333, 2...",0,"[46, 1670, 1826, 37, 1632, 46, 1991, 1525, 152...",[14864]


In [53]:
def convert_to_string(df):
  total_list = []
  for i in df:
    per_instance = []
    for j in i:
      per_instance.append(str(j))
    total_list.append(per_instance)
  return total_list

# word to vec title model
string_title = convert_to_string(df_train['title'])
# Word2Vec
model_title = Word2Vec(string_title, min_count=1)
print(model_title)

# word to vec abstract model
string_abstract = convert_to_string(df_train['abstract'])
# Word2Vec
model_abstract = Word2Vec(string_abstract, min_count=1)
print(model_abstract)


# word to vec coauthor model
string_coauthor = convert_to_string(df_train['authors'])

model_coauthor = Word2Vec(string_coauthor, min_count=1)
print(model_coauthor)








Word2Vec(vocab=3444, size=100, alpha=0.025)
Word2Vec(vocab=4845, size=100, alpha=0.025)
Word2Vec(vocab=20433, size=100, alpha=0.025)


## title and abstract word to embedding

In [102]:
# # title embedding
# prolific_author_string_title = convert_to_string(df_prolific_authors['title'])
# title_embedding = []
# for i in tqdm(prolific_author_string_title):
#   temp = np.zeros((len(i),100))
#   for k, j in enumerate(i):
#     temp[k,:] = model_title[j]
#   title_embedding.append(temp.mean(axis=0))


# # abstract embedding
# prolific_author_string_abstract = convert_to_string(df_prolific_authors['abstract'])
# abstract_embedding = []
# for i in tqdm(prolific_author_string_abstract):
#   temp = np.zeros((len(i),100))
#   for k, j in enumerate(i):
#     temp[k,:] = model_abstract[j]
#   abstract_embedding.append(temp.mean(axis=0))





# coauthor embedding
prolific_author_string_coauthor = convert_to_string(df_prolific_authors['coauthors'])
coauthor_embedding = []
for i in tqdm(prolific_author_string_coauthor):
  temp = np.zeros((len(i),100))
  for k, j in enumerate(i):
    temp[k,:] = model_coauthor[j]
  coauthor_embedding.append(temp.mean(axis=0))








  0%|          | 0/7460 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [103]:
# tfidf for abstract and title
def convert_to_instance_string(df):
  total_list = []
  for i in df:
    total_list.append(str(i)[1:-1].replace(",", ""))
  return total_list

# Counts for abstract
abstract_voc = {}
abstract_num = 4999
for i in range(abstract_num):
  abstract_voc[str(i)] = i




prolific_author_string_abstract = convert_to_instance_string(df_prolific_authors['abstract'])
vectorizer = TfidfVectorizer(vocabulary = abstract_voc)
abstract_tfidf = vectorizer.fit_transform(prolific_author_string_abstract)




# Counts for title
title_voc = {}
title_num = 4999
for i in range(title_num):
  title_voc[str(i)] = i




prolific_author_string_title = convert_to_instance_string(df_prolific_authors['title'])
vectorizer = TfidfVectorizer(vocabulary = title_voc)
title_tfidf = vectorizer.fit_transform(prolific_author_string_title)


abstract_tfidf = abstract_tfidf.toarray()
title_tfidf = title_tfidf.toarray()

## Truncated svd for tfidf abstract and title

In [99]:
svd = TruncatedSVD(n_components=100)
svd_tfidf_abstract = svd.fit_transform(abstract_tfidf)
svd_tfidf_title = svd.fit_transform(title_tfidf)


In [58]:

coauthors_features_name = ['coauthors_' + str(i) for i in range(100)]
# coauthors_features = np.vstack([coauthors_features_name,coauthors_ohe])
df_coauthors_features = pd.DataFrame(coauthor_embedding)
df_coauthors_features = df_coauthors_features.astype('float')
df_coauthors_features.columns = coauthors_features_name

# abstract_features_name = ['abstract_' + str(i) for i in range(100)]
# df_abstract_features = pd.DataFrame(abstract_embedding)
# df_abstract_features = df_abstract_features.astype('float')
# df_abstract_features.columns = abstract_features_name

# title_features_name = ['title_' + str(i) for i in range(100)]
# df_title_features = pd.DataFrame(title_embedding)
# df_title_features = df_title_features.astype('float')
# df_title_features.columns = title_features_name






## svd tfidf abstract and title

In [104]:
# tfidf features


abstract_features_name = ['abstract_' + str(i) for i in range(100)]
df_abstract_features = pd.DataFrame(svd_tfidf_abstract)
df_abstract_features = df_abstract_features.astype('float')
df_abstract_features.columns = abstract_features_name

title_features_name = ['title_' + str(i) for i in range(100)]
df_title_features = pd.DataFrame(svd_tfidf_title)
df_title_features = df_title_features.astype('float')
df_title_features.columns = title_features_name


In [106]:
df_merge_coauthors_sentences = pd.concat([df_coauthors_features, df_title_features,df_abstract_features], axis = 1)

In [107]:
df_merge_coauthors_sentences

,coauthors_0,coauthors_1,coauthors_2,coauthors_3,coauthors_4,coauthors_5,coauthors_6,coauthors_7,coauthors_8,coauthors_9,...,abstract_90,abstract_91,abstract_92,abstract_93,abstract_94,abstract_95,abstract_96,abstract_97,abstract_98,abstract_99
0,0.004070,-0.000615,0.002603,-0.003921,-0.000903,0.002468,0.003588,0.003299,0.000838,-0.002169,...,-0.028390,-0.001750,-0.031881,0.069309,0.044846,-0.005791,-0.045179,0.016284,-0.045886,0.035955
1,0.001978,-0.000452,-0.001113,0.000789,0.000188,0.000478,0.001330,0.000046,-0.002208,0.004257,...,0.003960,-0.048694,-0.007030,0.050816,-0.078684,-0.001301,-0.005036,0.019350,0.011077,0.050899
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020613,-0.032903,-0.057120,-0.025545,-0.022526,-0.049358,0.004396,0.013084,0.005122,-0.024782
3,-0.004500,-0.001009,0.004264,-0.002823,0.004300,-0.003201,0.004693,-0.004108,-0.003003,0.003518,...,0.018212,-0.029650,0.007407,-0.022627,-0.055866,0.011548,0.001693,-0.027182,-0.029480,-0.028052
4,-0.001455,-0.000591,0.005153,-0.000401,0.002030,0.003029,0.001058,-0.002050,-0.000522,0.001253,...,0.064992,0.030499,0.054257,0.027359,-0.010664,-0.052120,0.007989,0.081163,-0.027986,-0.034248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,0.003362,0.000635,0.002156,-0.000121,-0.000393,0.000424,0.001620,-0.000601,0.000029,-0.000818,...,-0.038988,-0.020383,-0.012243,-0.034549,-0.038222,-0.014772,-0.017174,0.020690,0.019450,-0.014123
7456,0.000811,0.000782,-0.001464,0.001073,0.000537,-0.002456,0.000169,-0.002429,-0.000412,-0.000638,...,-0.035875,0.007048,-0.093159,-0.015353,-0.013695,0.008209,-0.048073,0.040279,0.015303,0.038254
7457,-0.001988,0.000964,0.001844,-0.002248,-0.002633,0.003100,-0.001128,-0.000083,-0.000345,-0.001722,...,-0.024576,-0.015080,-0.002422,0.064899,-0.018330,0.024674,0.009734,-0.024327,-0.004913,0.037999
7458,0.002472,0.002238,-0.005151,-0.003259,-0.002853,0.002780,-0.002008,-0.004241,-0.004481,0.002963,...,0.051399,0.049730,-0.021060,0.014482,-0.046197,0.021938,-0.048803,0.006225,0.025657,-0.050576


In [108]:
df_coauthors_features

,coauthors_0,coauthors_1,coauthors_2,coauthors_3,coauthors_4,coauthors_5,coauthors_6,coauthors_7,coauthors_8,coauthors_9,...,coauthors_90,coauthors_91,coauthors_92,coauthors_93,coauthors_94,coauthors_95,coauthors_96,coauthors_97,coauthors_98,coauthors_99
0,0.004070,-0.000615,0.002603,-0.003921,-0.000903,0.002468,0.003588,0.003299,0.000838,-0.002169,...,0.003568,-0.000844,-0.004268,0.001711,0.004229,0.004648,0.002754,0.002204,-0.004706,0.002085
1,0.001978,-0.000452,-0.001113,0.000789,0.000188,0.000478,0.001330,0.000046,-0.002208,0.004257,...,-0.001083,-0.002123,0.002187,-0.002084,0.000836,-0.001165,-0.000575,0.000409,0.001259,0.004387
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-0.004500,-0.001009,0.004264,-0.002823,0.004300,-0.003201,0.004693,-0.004108,-0.003003,0.003518,...,0.002388,0.004363,-0.001527,-0.000125,0.001735,-0.001821,0.004639,-0.004982,0.001889,0.001525
4,-0.001455,-0.000591,0.005153,-0.000401,0.002030,0.003029,0.001058,-0.002050,-0.000522,0.001253,...,-0.001956,-0.002875,-0.000905,0.000543,0.003520,0.001619,0.001780,0.000360,-0.003687,0.002819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,0.003362,0.000635,0.002156,-0.000121,-0.000393,0.000424,0.001620,-0.000601,0.000029,-0.000818,...,0.001743,0.000618,-0.000435,-0.001415,-0.000242,0.001352,-0.000847,0.000791,-0.004100,0.001933
7456,0.000811,0.000782,-0.001464,0.001073,0.000537,-0.002456,0.000169,-0.002429,-0.000412,-0.000638,...,-0.000480,-0.003705,-0.000761,-0.003162,0.000271,0.001220,0.001046,0.001700,-0.000169,-0.001460
7457,-0.001988,0.000964,0.001844,-0.002248,-0.002633,0.003100,-0.001128,-0.000083,-0.000345,-0.001722,...,0.002798,-0.002419,-0.001859,-0.000496,-0.002760,0.001759,-0.000440,0.001103,0.001804,0.001368
7458,0.002472,0.002238,-0.005151,-0.003259,-0.002853,0.002780,-0.002008,-0.004241,-0.004481,0.002963,...,-0.002786,0.002163,-0.001726,0.000388,0.004177,0.001221,0.002600,-0.003138,-0.001744,-0.002430


In [109]:
df_title_features

,title_0,title_1,title_2,title_3,title_4,title_5,title_6,title_7,title_8,title_9,...,title_90,title_91,title_92,title_93,title_94,title_95,title_96,title_97,title_98,title_99
0,0.254859,-0.108570,0.084303,0.043708,-0.028017,0.043174,0.064343,0.096756,-0.005888,0.007088,...,0.012568,0.070860,0.065105,0.079782,0.059845,-0.056684,-0.015170,0.062850,-0.050457,-0.085615
1,0.214361,-0.041040,-0.057099,-0.111725,0.051652,-0.048697,-0.211987,-0.092128,0.131879,0.116723,...,0.053050,-0.006931,0.027368,-0.029886,0.072034,-0.047041,0.044510,0.034466,-0.071138,0.009367
2,0.205286,-0.084554,-0.055899,-0.023346,-0.254038,0.051299,0.087659,-0.013031,0.033980,-0.046579,...,0.052158,0.046858,-0.064569,0.062709,-0.081264,0.046781,0.019878,-0.023722,0.003838,0.017345
3,0.228298,0.278572,-0.015200,0.026369,-0.036606,0.035079,-0.058701,0.028394,-0.027657,-0.042835,...,-0.059226,-0.000045,0.000002,-0.099358,0.040872,-0.023331,0.019633,0.021169,0.035827,0.026362
4,0.343190,-0.059906,0.013079,-0.174380,-0.003504,-0.125779,-0.004013,0.166421,0.157912,0.295942,...,0.030843,0.057527,0.022133,-0.068600,0.097137,0.026848,-0.039718,-0.064477,-0.022549,-0.002810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,0.211992,-0.076684,-0.007903,0.002297,-0.064822,0.027958,0.049189,-0.129191,-0.244316,0.113941,...,-0.018443,0.010302,0.019531,0.045590,0.041290,0.001638,-0.063247,-0.054144,0.034369,-0.008843
7456,0.206939,-0.081765,-0.093233,-0.096270,-0.106725,-0.039641,-0.104139,0.109473,0.013359,-0.117138,...,0.056379,0.029593,0.084212,-0.131581,-0.012515,-0.046886,-0.021887,-0.089770,0.061676,-0.023275
7457,0.076371,-0.025782,0.006001,-0.043021,-0.037799,-0.009858,-0.023466,-0.011372,0.063175,0.076600,...,0.001675,0.011306,0.034119,-0.049027,0.023352,0.023867,-0.047429,-0.042405,-0.077952,-0.013366
7458,0.268651,-0.068963,-0.094552,0.020677,-0.137853,-0.011087,0.059791,0.000526,0.135944,-0.056729,...,-0.047927,-0.148011,0.043917,-0.024893,0.068318,0.062070,0.033572,-0.007390,-0.021527,-0.024424


In [110]:
df_abstract_features

,abstract_0,abstract_1,abstract_2,abstract_3,abstract_4,abstract_5,abstract_6,abstract_7,abstract_8,abstract_9,...,abstract_90,abstract_91,abstract_92,abstract_93,abstract_94,abstract_95,abstract_96,abstract_97,abstract_98,abstract_99
0,0.252152,0.004069,-0.053475,0.010435,-0.028095,0.131946,0.035335,0.069309,0.002218,-0.018228,...,-0.028390,-0.001750,-0.031881,0.069309,0.044846,-0.005791,-0.045179,0.016284,-0.045886,0.035955
1,0.313118,0.018364,0.033630,0.248633,-0.039164,0.027788,-0.029941,-0.038769,0.044417,-0.016472,...,0.003960,-0.048694,-0.007030,0.050816,-0.078684,-0.001301,-0.005036,0.019350,0.011077,0.050899
2,0.260993,-0.075531,0.003495,-0.038955,0.012610,-0.029614,-0.040717,0.009340,-0.013031,0.031034,...,0.020613,-0.032903,-0.057120,-0.025545,-0.022526,-0.049358,0.004396,0.013084,0.005122,-0.024782
3,0.257214,0.059133,0.072577,-0.066294,-0.053368,0.079129,-0.023745,0.077251,-0.033194,-0.026067,...,0.018212,-0.029650,0.007407,-0.022627,-0.055866,0.011548,0.001693,-0.027182,-0.029480,-0.028052
4,0.397995,0.081086,-0.242015,-0.005949,0.015757,-0.041401,-0.049957,-0.090917,-0.030035,0.060745,...,0.064992,0.030499,0.054257,0.027359,-0.010664,-0.052120,0.007989,0.081163,-0.027986,-0.034248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,0.390574,-0.006049,0.167429,0.179404,-0.097047,0.006521,0.021474,-0.028245,0.043008,-0.053072,...,-0.038988,-0.020383,-0.012243,-0.034549,-0.038222,-0.014772,-0.017174,0.020690,0.019450,-0.014123
7456,0.344729,-0.077571,0.047851,0.000627,-0.109016,0.006301,-0.070687,0.072201,-0.023332,-0.038327,...,-0.035875,0.007048,-0.093159,-0.015353,-0.013695,0.008209,-0.048073,0.040279,0.015303,0.038254
7457,0.243634,-0.058212,0.067023,0.068566,-0.013297,-0.057962,-0.030745,-0.002243,0.001565,0.001352,...,-0.024576,-0.015080,-0.002422,0.064899,-0.018330,0.024674,0.009734,-0.024327,-0.004913,0.037999
7458,0.256890,-0.054611,-0.034869,-0.045049,-0.015139,-0.012969,-0.041087,0.055144,-0.053973,0.041404,...,0.051399,0.049730,-0.021060,0.014482,-0.046197,0.021938,-0.048803,0.006225,0.025657,-0.050576


In [111]:
df_merge_coauthors_sentences.fillna(0,inplace =True)
df_coauthors_features.fillna(0,inplace =True)

In [199]:
X_train,X_test,y_train,y_test = train_test_split(df_coauthors_features,ohe_prolific_labels,test_size=0.1,random_state=90051)

## one versus rest model

In [158]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test= scaler.transform(X_test)
base_lr = LogisticRegression(solver='liblinear', random_state=0,max_iter = 400)
clf = OneVsRestClassifier(base_lr).fit(X_train, y_train)
predictions = clf.predict(X_test)
f1_score(y_test, predictions, average=None)

array([0.78947368, 0.5       , 0.28571429, 0.22222222, 0.61538462,
       0.28571429, 0.52173913, 0.70588235, 0.33333333, 0.4       ,
       0.375     , 0.58823529, 0.1875    , 0.4       , 0.88888889,
       0.54545455, 0.4       , 0.46153846, 0.4       , 0.66666667,
       0.        , 0.5       , 0.36363636, 0.375     , 0.        ,
       0.43478261, 0.47058824, 0.38095238, 0.26666667, 0.625     ,
       0.        , 0.4       , 0.62068966, 0.8       , 0.44444444,
       0.5       , 0.        , 0.45454545, 0.54545455, 0.23529412,
       0.53333333, 0.33333333, 0.4       , 0.85714286, 0.6       ,
       0.71428571, 0.22222222, 0.70967742, 0.        , 0.5       ,
       0.28571429, 0.28571429, 0.22222222, 0.36842105, 0.        ,
       0.90909091, 0.42857143, 0.73684211, 0.53333333, 0.63157895,
       0.66666667, 0.70588235, 0.2       , 0.66666667, 0.33333333,
       0.56      , 0.57142857, 0.4       , 0.28571429, 0.30769231,
       0.66666667, 0.33333333, 0.52631579, 0.22222222, 0.38461

In [159]:
f1_score(y_test, predictions, average='samples')

0.40389378271415804

## classifierchain model (目前最好的model) Logistic Regression

In [172]:
base_lr = LogisticRegression(solver='liblinear', random_state=0,max_iter = 200)
chain = ClassifierChain(base_lr, order='random', random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test= scaler.transform(X_test)
chain.fit(X_scaled_train, y_train)
predictions = chain.predict(X_scaled_test)
f1_score(y_test, predictions, average=None)

array([0.81081081, 0.5       , 0.28571429, 0.22222222, 0.61538462,
       0.30769231, 0.5       , 0.70588235, 0.33333333, 0.25      ,
       0.4       , 0.66666667, 0.12903226, 0.42105263, 0.75      ,
       0.5625    , 0.48275862, 0.46153846, 0.33333333, 0.76923077,
       0.        , 0.5       , 0.36363636, 0.47058824, 0.        ,
       0.43478261, 0.47058824, 0.45454545, 0.35294118, 0.66666667,
       0.        , 0.4       , 0.62068966, 0.8       , 0.54545455,
       0.5       , 0.15384615, 0.45454545, 0.54545455, 0.23529412,
       0.53333333, 0.35294118, 0.4       , 0.85714286, 0.66666667,
       0.76923077, 0.22222222, 0.66666667, 0.        , 0.5       ,
       0.28571429, 0.28571429, 0.22222222, 0.36842105, 0.        ,
       0.90909091, 0.42857143, 0.73684211, 0.53333333, 0.7       ,
       0.66666667, 0.70588235, 0.22222222, 0.66666667, 0.18181818,
       0.52173913, 0.53333333, 0.53333333, 0.28571429, 0.30769231,
       0.66666667, 0.2       , 0.47058824, 0.22222222, 0.41666

In [173]:
f1_score(y_test, predictions, average='samples')

0.4126899016979445

## classifierchain model (目前最好的model) MLP

In [211]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test= scaler.transform(X_test)

clf = MLPClassifier(random_state=0, max_iter=300, verbose = True,hidden_layer_sizes=(256,128))

clf.fit(X_scaled_train, y_train)
predictions = clf.predict(X_scaled_test)
f1_score(y_test, predictions, average=None)

Iteration 1, loss = 30.38368044
Iteration 2, loss = 8.12413282
Iteration 3, loss = 7.20533972
Iteration 4, loss = 6.77303814
Iteration 5, loss = 6.36506786
Iteration 6, loss = 5.94366730
Iteration 7, loss = 5.52396288
Iteration 8, loss = 5.12210064
Iteration 9, loss = 4.76204830
Iteration 10, loss = 4.44803811
Iteration 11, loss = 4.17795969
Iteration 12, loss = 3.94258492
Iteration 13, loss = 3.73836160
Iteration 14, loss = 3.56144183
Iteration 15, loss = 3.40317907
Iteration 16, loss = 3.25964662
Iteration 17, loss = 3.12963186
Iteration 18, loss = 3.00954470
Iteration 19, loss = 2.89845853
Iteration 20, loss = 2.79457399
Iteration 21, loss = 2.69660629
Iteration 22, loss = 2.59960156
Iteration 23, loss = 2.51630887
Iteration 24, loss = 2.43316168
Iteration 25, loss = 2.35304942
Iteration 26, loss = 2.27371752
Iteration 27, loss = 2.20152159
Iteration 28, loss = 2.13193864
Iteration 29, loss = 2.06597998
Iteration 30, loss = 2.00678125
Iteration 31, loss = 1.94757406
Iteration 32, lo

array([0.97142857, 0.5       , 0.5       , 0.28571429, 0.54545455,
       0.54545455, 0.54545455, 0.77777778, 0.4       , 0.5       ,
       0.78787879, 0.75      , 0.48      , 0.7826087 , 0.61538462,
       0.72      , 0.6       , 0.71428571, 0.28571429, 0.83333333,
       0.28571429, 0.8       , 0.61538462, 0.35294118, 0.4       ,
       0.52631579, 0.64516129, 0.71428571, 0.54545455, 0.75      ,
       0.        , 0.66666667, 0.66666667, 0.81818182, 0.61538462,
       0.71428571, 0.44444444, 0.66666667, 0.54545455, 0.63157895,
       0.70588235, 0.52631579, 0.66666667, 0.6       , 0.33333333,
       0.71428571, 0.28571429, 0.64516129, 0.54545455, 0.66666667,
       0.88888889, 0.        , 0.28571429, 0.8       , 0.22222222,
       0.88888889, 0.28571429, 0.55555556, 0.625     , 0.4       ,
       0.        , 0.61538462, 0.66666667, 0.76923077, 0.5       ,
       0.78571429, 0.57142857, 0.4       , 0.5       , 0.66666667,
       0.63157895, 0.4       , 0.82352941, 0.75      , 0.47619

In [212]:
f1_score(y_test, predictions, average='samples')

0.5347504149112727

## MultiOutputClassifier

In [176]:
base_lr = LogisticRegression(solver='liblinear', random_state=0,max_iter = 200)
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test= scaler.transform(X_test)
clf = MultiOutputClassifier(base_lr).fit(X_scaled_train, y_train)
predictions = clf.predict(X_scaled_test)
f1_score(y_test, predictions, average=None)

array([0.78947368, 0.5       , 0.28571429, 0.22222222, 0.61538462,
       0.28571429, 0.52173913, 0.70588235, 0.33333333, 0.4       ,
       0.375     , 0.58823529, 0.1875    , 0.4       , 0.88888889,
       0.54545455, 0.4       , 0.46153846, 0.4       , 0.66666667,
       0.        , 0.5       , 0.36363636, 0.375     , 0.        ,
       0.43478261, 0.47058824, 0.38095238, 0.26666667, 0.625     ,
       0.        , 0.4       , 0.62068966, 0.8       , 0.44444444,
       0.5       , 0.        , 0.45454545, 0.54545455, 0.23529412,
       0.53333333, 0.33333333, 0.4       , 0.85714286, 0.6       ,
       0.71428571, 0.22222222, 0.70967742, 0.        , 0.5       ,
       0.28571429, 0.28571429, 0.22222222, 0.36842105, 0.        ,
       0.90909091, 0.42857143, 0.73684211, 0.53333333, 0.63157895,
       0.66666667, 0.70588235, 0.2       , 0.66666667, 0.33333333,
       0.56      , 0.57142857, 0.4       , 0.28571429, 0.30769231,
       0.66666667, 0.33333333, 0.52631579, 0.22222222, 0.38461

In [178]:
f1_score(y_test, predictions, average='samples')

0.40389378271415804

## Test set

In [ ]:
#Test
# Counts for sentences
sentences_voc = {}
sentences_num = 4999
for i in range(sentences_num):
  sentences_voc[str(i)] = i

test_string_sentences = convert_to_string(df_test['sentences'])
test_sentences_vectorizer = CountVectorizer(vocabulary = sentences_voc)
test_sentences_counts = test_sentences_vectorizer.fit_transform(test_string_sentences).toarray()  

# Counts for coauthors
coauthor_voc = {}
coauthor_num = 21246
for i in range(coauthor_num):
  coauthor_voc[str(i)] = i

test_string_coauthor = convert_to_string(df_test['coauthors'])
test_coauthor_vectorizer = CountVectorizer(vocabulary = coauthor_voc)
test_coauthor_counts = coauthor_vectorizer.fit_transform(test_string_coauthor).toarray()



test_coauthors_features_name = ['coauthors_' + str(i) for i in range(21246)]
# coauthors_features = np.vstack([coauthors_features_name,coauthors_ohe])
df_test_coauthors_features = pd.DataFrame(test_coauthor_counts)
df_test_coauthors_features = df_test_coauthors_features.astype('float')
df_test_coauthors_features.columns = test_coauthors_features_name

test_sentence_features_name = ['sentences_' + str(i) for i in range(1,5000)]
df_test_sentence_features = pd.DataFrame(test_sentences_counts)
df_test_sentence_features = df_test_sentence_features.astype('float')
df_test_sentence_features.columns = test_sentence_features_name


df_test_merge_coauthors_sentences = pd.concat([df_test_coauthors_features, df_test_sentence_features], axis = 1)




In [ ]:
df_test_merge_coauthors_sentences

In [ ]:
X_scaled_predict= scaler.transform(df_test_merge_coauthors_sentences)

In [ ]:
predition = chain.predict(X_scaled_predict)

In [ ]:
predition

In [ ]:
predicted_list = [i.tolist() for i in predition]
total_index_list = []
for i in predicted_list:
  index_list = []
  for j,k in enumerate(i):
    if k == 1:
      index_list.append(str(j))
  if len(index_list) != 0:
    total_index_list.append(index_list)
  else:
    total_index_list.append([str(-1)])

predicted_str = []
for i in total_index_list:
  a = ','.join(i)
  predicted_str.append(a)

In [ ]:

df_output = pd.DataFrame({'ID': list(range(800)),'Predict':predicted_str})
df_output.to_csv('predicted_result.csv',index = False)